In [1]:
from flask import Flask, request, url_for, redirect, render_template, jsonify, make_response, session
from flask_mysqldb import MySQL
import MySQLdb.cursors
import numpy as np
import re

In [2]:
app = Flask(__name__)

# Secret key (security)
app.secret_key = 'performancedashboard'

app.config['MYSQL_USER'] = 'sql4421674'
app.config['MYSQL_PASSWORD'] = 'J5NwZ4R8xY'
app.config['MYSQL_HOST'] = 'sql4.freemysqlhosting.net'
app.config['MYSQL_DB'] = 'sql4421674'
app.config['MYSQL_CURSORCLASS'] = 'DictCursor'

mysql = MySQL(app)

In [3]:
@app.route('/', methods=['GET', 'POST'])
def login():
    # Output message if something goes wrong...
    error = None
    # Check if "username" and "password" POST requests exist (user submitted form)
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form:
        # Create variables for easy access
        username = request.form['username']
        password = request.form['password']
        # Check if account exists using MySQL
        cursor = mysql.connection.cursor(MySQLdb.cursors.DictCursor)
        cursor.execute('SELECT * FROM executives WHERE username = %s AND password = %s', (username, password))
        # Fetch one record and return result
        account = cursor.fetchone()
        # If account exists in accounts table in out database
        if account:
            # Create session data, we can access this data in other routes
            session['loggedin'] = True
            session['id'] = account['id']
            session['username'] = account['username']
            # Redirect to home page
            return redirect(url_for('home'))
        
        else:
            # Account doesnt exist or username/password incorrect
            error = 'Incorrect username/password!'
        
    # Show the login form with message (if any)
    return render_template('login.html', error=error)

In [4]:
@app.route('/home')
def home():
    # Check if user is loggedin
    if 'loggedin' in session:
        # User is loggedin show them the home page
        return render_template('index.html', username=session['username'])

In [5]:
@app.route('/resetpassword', methods=['GET', 'POST'])
def password():
    msg = None
    
    if request.method == 'POST' and 'username' in request.form and 'password' in request.form and 'new password' in request.form:
        # Create variables for easy access
        username = request.form['username']
        password = request.form['password']
        newpassword = request.form['new password']
        
        # Check if account exists using MySQL
        cursor = mysql.connection.cursor()
        cursor.execute('SELECT * FROM executives WHERE username = %s AND password = %s', (username, password))
        
        account = cursor.fetchone()
        
        # If account exists in accounts table in out database
        if account:
            cursor.execute('UPDATE executives SET password = %s  WHERE username = %s', (newpassword, username))
            mysql.connection.commit()
            return redirect(url_for('login'))
        else:
            msg="Username/password is wrong"
    
    return render_template('changepass.html', msg=msg)

In [6]:
@app.route('/logout')
def logout():
    # Remove session data, this will log the user out
    session.pop('loggedin', None)
    session.pop('id', None)
    session.pop('username', None)
    # Redirect to login page
    return redirect(url_for('login'))

In [7]:
@app.route('/dashboard')
def dashboard():
    # show the form, it wasn't submitted
    return render_template('dashboard.html')

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jun/2021 17:01:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2021 17:01:51] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [27/Jun/2021 17:01:51] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2021 17:01:51] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Jun/2021 17:01:52] "GET /dashboard HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2021 17:01:52] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Jun/2021 17:02:03] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2021 17:02:03] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Jun/2021 17:02:06] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [27/Jun/2021 17:02:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2021 17:02:07] "GET /resetpassword HTTP/1.1" 200 -
